# Benchmark the reliability solvers on the problems

In this example, we show how to run all the methods on all the problems and get the computed probability.

In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd
import csv
from tqdm import tqdm
from IPython.display import display, HTML

We import the list of reliability problems.

In [2]:
benchmarkProblemList = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(benchmarkProblemList)
numberOfProblems

26

For each problem in the benchmark, print the problem name and the exact failure probability.

In [3]:
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    pf = problem.getProbability()
    print("#", i, " : ", name, ", exact PF : ", pf)

# 0  :  RP8 , exact PF :  0.000784
# 1  :  RP14 , exact PF :  0.00752
# 2  :  RP22 , exact PF :  0.00416
# 3  :  RP24 , exact PF :  0.00286
# 4  :  RP25 , exact PF :  6.14e-06
# 5  :  RP28 , exact PF :  1.46e-07
# 6  :  RP31 , exact PF :  0.00018
# 7  :  RP33 , exact PF :  0.00257
# 8  :  RP35 , exact PF :  0.00354
# 9  :  RP38 , exact PF :  0.0081
# 10  :  RP53 , exact PF :  0.0313
# 11  :  RP55 , exact PF :  0.5600144282863704
# 12  :  RP54 , exact PF :  0.000998
# 13  :  RP57 , exact PF :  0.0284
# 14  :  RP75 , exact PF :  0.0107
# 15  :  RP89 , exact PF :  0.00543
# 16  :  RP107 , exact PF :  2.92e-07
# 17  :  RP110 , exact PF :  3.19e-05
# 18  :  RP111 , exact PF :  7.65e-07
# 19  :  RP63 , exact PF :  0.000379
# 20  :  RP91 , exact PF :  0.000697
# 21  :  RP60 , exact PF :  0.0456
# 22  :  RP77 , exact PF :  2.87e-07
# 23  :  Four-branch serial system , exact PF :  0.002185961454913241
# 24  :  R-S , exact PF :  0.07864960352514257
# 25  :  Axial stressed beam , exact PF :  0.02

## Run several algorithms on a single problem

We want to run several algorithms on a single problem. We set the parameters of the algorithms and run them on a single problem.

In [4]:
maximumEvaluationNumber = 1000
maximumAbsoluteError = 1.0e-3
maximumRelativeError = 1.0e-3
maximumResidualError = 1.0e-3
maximumConstraintError = 1.0e-3
nearestPointAlgorithm = ot.AbdoRackwitz()
nearestPointAlgorithm.setMaximumEvaluationNumber(maximumEvaluationNumber)
nearestPointAlgorithm.setMaximumAbsoluteError(maximumAbsoluteError)
nearestPointAlgorithm.setMaximumRelativeError(maximumRelativeError)
nearestPointAlgorithm.setMaximumResidualError(maximumResidualError)
nearestPointAlgorithm.setMaximumConstraintError(maximumConstraintError)

In [5]:
i = 3
problem = benchmarkProblemList[i]
metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)

We try the FORM algorithm. 

In [6]:
benchmarkFORM = metaAlgorithm.runFORM(nearestPointAlgorithm)
s1 = benchmarkFORM.summary()
print(s1)

computedProbability = 0.006209245091320783
exactProbability = 0.00286
absoluteError = 0.003349245091320783
numberOfCorrectDigits = -0.06858089640947408
numberOfFunctionEvaluations = 5
numberOfDigitsPerEvaluation = -0.013716179281894817


Then the SORM algorithm. 

In [7]:
benchmarkSORM = metaAlgorithm.runSORM(nearestPointAlgorithm)
s2 = benchmarkSORM.summary()
print(s2)

computedProbability = 0.006209245091320783
exactProbability = 0.00286
absoluteError = 0.003349245091320783
numberOfCorrectDigits = -0.06858089640947408
numberOfFunctionEvaluations = 5
numberOfDigitsPerEvaluation = -0.013716179281894817


In [8]:
benchmarkMC = metaAlgorithm.runMonteCarlo(
    maximumOuterSampling=1000000, coefficientOfVariation=0.1, blockSize=1,
)
s3 = benchmarkMC.summary()
print(s3)

computedProbability = 0.002858694719990857
exactProbability = 0.00286
absoluteError = 1.3052800091430984e-06
numberOfCorrectDigits = 3.340662346449938
numberOfFunctionEvaluations = 34981
numberOfDigitsPerEvaluation = 9.549933811068688e-05


In [9]:
benchmarkFORMIS = metaAlgorithm.runFORMImportanceSampling(
    nearestPointAlgorithm,
    maximumOuterSampling=1000,
    coefficientOfVariation=0.1,
    blockSize=1,
)
s4 = benchmarkFORMIS.summary()
print(s4)

computedProbability = 0.0028912458245778103
exactProbability = 0.00286
absoluteError = 3.124582457781017e-05
numberOfCorrectDigits = 1.9615740429360635
numberOfFunctionEvaluations = 601
numberOfDigitsPerEvaluation = 0.003263850321025064


In [10]:
benchmarkSS = metaAlgorithm.runSubsetSampling(
    maximumOuterSampling=5000, coefficientOfVariation=0.1, blockSize=1,
)
s5 = benchmarkSS.summary()
print(s5)

computedProbability = 0.002817999999999997
exactProbability = 0.00286
absoluteError = 4.200000000000297e-05
numberOfCorrectDigits = 1.8331167427311117
numberOfFunctionEvaluations = 15000
numberOfDigitsPerEvaluation = 0.0001222077828487408


## Run all algorithms on all problems and produce a single result table

For several algorithms and all the reliability problems, we want to estimate the failure probability and compare them.

We create a list of problem names. 

In [11]:
problem_names = []
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    problem_names.append(name)

In [12]:
metrics = [
    "Exact",
    "FORM",
    "SORM",
    "Monte Carlo",
    "FORM-IS",
    "Subset",
]
results = np.zeros((numberOfProblems, len(metrics)))
maximumOuterSampling = 10 ** 2
blockSize = 10 ** 2
coefficientOfVariation = 0.0

for i in tqdm(range(numberOfProblems)):
    problem = benchmarkProblemList[i]
    results[i][0] = problem.getProbability()
    metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)
    benchmarkResult = metaAlgorithm.runFORM(nearestPointAlgorithm)
    results[i][1] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runSORM(nearestPointAlgorithm)
    results[i][2] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runMonteCarlo(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[i][3] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runFORMImportanceSampling(
        nearestPointAlgorithm,
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[i][4] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runSubsetSampling(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[i][5] = benchmarkResult.computedProbability

df = pd.DataFrame(results, index=problem_names, columns=metrics)
df

 38%|███▊      | 10/26 [00:04<00:06,  2.30it/s]/home/devel/GIT/otbenchmark/otbenchmark/ReliabilityLibrary.py:47: RuntimeWarning: divide by zero encountered in log
  logRelativeError = -np.log(relativeError) / np.log(basis)
100%|██████████| 26/26 [00:09<00:00,  2.61it/s]


,Exact,FORM,SORM,Monte Carlo,FORM-IS,Subset
RP8,7.840000e-04,6.598878e-04,7.838036e-04,0.0004,7.996083e-04,9.154000e-04
RP14,7.520000e-03,7.003011e-04,6.995436e-04,0.0010,7.735961e-04,7.201000e-04
RP22,4.160000e-03,6.209672e-03,4.390902e-03,0.0043,4.313709e-03,3.899000e-03
RP24,2.860000e-03,6.209245e-03,6.209245e-03,0.0030,2.816315e-03,2.518000e-03
RP25,6.140000e-06,6.349316e-03,6.349316e-03,0.0001,4.476953e-05,3.874122e-05
RP28,1.460000e-07,2.850470e-08,0.000000e+00,0.0000,1.263642e-07,1.319034e-07
RP31,1.800000e-04,2.275013e-02,2.275013e-02,0.0033,2.993455e-03,2.940000e-03
RP33,2.570000e-03,1.349898e-03,1.349898e-03,0.0032,2.511609e-03,2.489508e-03
RP35,3.540000e-03,1.349898e-03,2.134376e-03,0.0037,2.331039e-03,3.964000e-03
RP38,8.100000e-03,7.902212e-03,8.029356e-03,0.0064,8.154653e-03,8.229000e-03


In [13]:
df.to_csv("reliability_benchmark_table-output.csv")

## Run several algorithms on all problems and get detailed statistics

Run several algorithms on all reliability benchmark problems: print statistics on each problem.

In [14]:
def FormatRow(benchmarkResult):
    """Format a single row of the benchmark table"""
    result = [
        benchmarkResult.exactProbability,
        benchmarkResult.computedProbability,
        benchmarkResult.absoluteError,
        benchmarkResult.numberOfCorrectDigits,
        benchmarkResult.numberOfFunctionEvaluations,
        benchmarkResult.numberOfDigitsPerEvaluation,
    ]
    return result

In [15]:
method_names = ["Monte-Carlo", "FORM", "SORM", "FORM-IS", "SUBSET"]

maximumOuterSampling = 10 ** 1
blockSize = 10 ** 1
coefficientOfVariation = 0.0

for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem_names[i]
    exact_pf_name = "%10s" % ("Exact PF " + name[0:10])
    metrics = [
        exact_pf_name,
        "Estimated PF",
        "Absolute Error",
        "Correct Digits",
        "Function Calls",
        "Digits / Evaluation",
    ]
    results = np.zeros((len(method_names), len(metrics)))
    metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)
    # Monte-Carlo
    benchmarkResult = metaAlgorithm.runMonteCarlo(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[0, :] = FormatRow(benchmarkResult)
    # FORM
    benchmarkResult = metaAlgorithm.runFORM(nearestPointAlgorithm)
    results[1, :] = FormatRow(benchmarkResult)
    # SORM
    benchmarkResult = metaAlgorithm.runSORM(nearestPointAlgorithm)
    results[2, :] = FormatRow(benchmarkResult)
    # FORM-IS
    benchmarkResult = metaAlgorithm.runFORMImportanceSampling(
        nearestPointAlgorithm,
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[3, :] = FormatRow(benchmarkResult)
    # Subset
    benchmarkResult = metaAlgorithm.runSubsetSampling(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[4, :] = FormatRow(benchmarkResult)
    # Gather statistics and print them
    df = pd.DataFrame(results, index=method_names, columns=metrics,)
    # Format the columns for readability
    s = df.style.format(
        {
            exact_pf_name: lambda x: "{:.3e}".format(x),
            "Estimated PF": lambda x: "{:.3e}".format(x),
            "Absolute Error": lambda x: "{:.3e}".format(x),
            "Correct Digits": lambda x: "{:.1f}".format(abs(x)),
            "Function Calls": lambda x: "{:d}".format(int(x)),
            "Estimated PF": lambda x: "{:.2e}".format(x),
            "Digits / Evaluation": lambda x: "{:.1f}".format(abs(x)),
        }
    )
    display(s)

,Exact PF RP8,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,7.840e-04,0.00e+00,7.840e-04,0.0,100,0.0
FORM,7.840e-04,6.60e-04,1.241e-04,0.8,7,0.1
SORM,7.840e-04,7.84e-04,1.964e-07,3.6,7,0.5
FORM-IS,7.840e-04,9.72e-04,1.883e-04,0.6,107,0.0
SUBSET,7.840e-04,9.50e-04,1.660e-04,0.7,400,0.0


,Exact PF RP14,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,7.520e-03,0.00e+00,7.520e-03,0.0,100,0.0
FORM,7.520e-03,7.00e-04,6.820e-03,0.0,12,0.0
SORM,7.520e-03,7.00e-04,6.820e-03,0.0,12,0.0
FORM-IS,7.520e-03,1.00e-03,6.520e-03,0.1,112,0.0
SUBSET,7.520e-03,9.50e-04,6.570e-03,0.1,400,0.0


,Exact PF RP22,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,4.160e-03,0.00e+00,4.160e-03,0.0,100,0.0
FORM,4.160e-03,6.21e-03,2.050e-03,0.3,5,0.1
SORM,4.160e-03,4.39e-03,2.309e-04,1.3,5,0.3
FORM-IS,4.160e-03,3.83e-03,3.299e-04,1.1,105,0.0
SUBSET,4.160e-03,2.90e-03,1.260e-03,0.5,300,0.0


,Exact PF RP24,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,2.860e-03,0.00e+00,2.860e-03,0.0,100,0.0
FORM,2.860e-03,6.21e-03,3.349e-03,0.1,5,0.0
SORM,2.860e-03,6.21e-03,3.349e-03,0.1,5,0.0
FORM-IS,2.860e-03,1.72e-03,1.141e-03,0.4,105,0.0
SUBSET,2.860e-03,7.90e-04,2.070e-03,0.1,400,0.0


,Exact PF RP25,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,6.140e-06,0.00e+00,6.140e-06,0.0,100,0.0
FORM,6.140e-06,6.35e-03,6.343e-03,3.0,1016,0.0
SORM,6.140e-06,6.35e-03,6.343e-03,3.0,1029,0.0
FORM-IS,6.140e-06,2.33e-05,1.716e-05,0.4,1116,0.0
SUBSET,6.140e-06,4.50e-05,3.886e-05,0.8,500,0.0


...

,Exact PF RP60,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,4.560e-02,5.00e-02,4.400e-03,1.0,100,0.0
FORM,4.560e-02,4.48e-02,7.603e-04,1.8,46,0.0
SORM,4.560e-02,4.48e-02,7.603e-04,1.8,107,0.0
FORM-IS,4.560e-02,3.18e-02,1.380e-02,0.5,146,0.0
SUBSET,4.560e-02,5.60e-02,1.040e-02,0.6,200,0.0


,Exact PF RP77,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,2.870e-07,0.00e+00,2.870e-07,0.0,100,0.0
FORM,2.870e-07,6.28e-03,6.276e-03,4.3,1012,0.0
SORM,2.870e-07,0.00e+00,2.870e-07,0.0,1037,0.0
FORM-IS,2.870e-07,0.00e+00,2.870e-07,0.0,1112,0.0
SUBSET,2.870e-07,1.44e-07,1.430e-07,0.3,700,0.0


,Exact PF Four-branc,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,2.186e-03,0.00e+00,2.186e-03,0.0,100,0.0
FORM,2.186e-03,0.00e+00,2.186e-03,0.0,5,0.0
SORM,2.186e-03,0.00e+00,2.186e-03,0.0,5,0.0
FORM-IS,2.186e-03,0.00e+00,2.186e-03,0.0,5,0.0
SUBSET,2.186e-03,3.30e-03,1.114e-03,0.3,300,0.0


,Exact PF R-S,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,7.865e-02,4.00e-02,3.865e-02,0.3,100,0.0
FORM,7.865e-02,7.86e-02,4.163e-17,15.3,3,5.1
SORM,7.865e-02,7.86e-02,4.163e-17,15.3,3,5.1
FORM-IS,7.865e-02,7.73e-02,1.389e-03,1.8,103,0.0
SUBSET,7.865e-02,5.70e-02,2.165e-02,0.6,200,0.0


,Exact PF Axial stre,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,2.920e-02,3.00e-02,8.018e-04,1.6,100,0.0
FORM,2.920e-02,3.00e-02,7.846e-04,1.6,5,0.3
SORM,2.920e-02,2.93e-02,1.344e-04,2.3,5,0.5
FORM-IS,2.920e-02,2.99e-02,6.759e-04,1.6,105,0.0
SUBSET,2.920e-02,2.20e-02,7.198e-03,0.6,200,0.0
